In this notebook, we analyze the `data/amends.csv`-file to find the account responsible for insider trading.

First, we import the `pandas` and `numpy`-libraries.

In [1]:
import pandas as pd
import numpy as np

Then we read the `csv`-file into a dataframe. We also divide the price by 100 to get the actual dollar values.

In [2]:
df = pd.read_csv('data/amends.csv')
df.price = df.price/100
df.head()

,account,direction,id,ts,price,qty
0,MSL19797553,buy,946,2016-03-30T18:47:54.733193387Z,33.79,220
1,MSL19797553,buy,946,2016-03-30T18:47:54.733193387Z,33.79,220
2,MSL19797553,buy,946,2016-03-30T18:47:54.735164602Z,33.79,366
3,MSL19797553,buy,946,2016-03-30T18:47:54.733193387Z,33.79,220
4,MSL19797553,buy,946,2016-03-30T18:47:54.735164602Z,33.79,366


To compute the net position, we generate a `pos_dir` dummy variable that denotes buying/selling.

In [3]:
df['pos_dir'] = np.where(df['direction'] == 'buy', 1, -1)
df['net_position'] = df['pos_dir']*df['qty']
df['net_cash'] = -1*df['pos_dir']*df['qty']*df['price']
df.head()

,account,direction,id,ts,price,qty,pos_dir,net_position,net_cash
0,MSL19797553,buy,946,2016-03-30T18:47:54.733193387Z,33.79,220,1,220,-7433.80
1,MSL19797553,buy,946,2016-03-30T18:47:54.733193387Z,33.79,220,1,220,-7433.80
2,MSL19797553,buy,946,2016-03-30T18:47:54.735164602Z,33.79,366,1,366,-12367.14
3,MSL19797553,buy,946,2016-03-30T18:47:54.733193387Z,33.79,220,1,220,-7433.80
4,MSL19797553,buy,946,2016-03-30T18:47:54.735164602Z,33.79,366,1,366,-12367.14


In perhaps the most crucial step, we group by account names, and perform a few aggregations:
 * the sum of net positions
 * the sum of net cash 
 * the total number of fills (here we compute the len of the direction, the variable does not matter)
 * and the number of unique orders (i.e. the number of unique ids).

In [4]:
agg = df.groupby('account').agg({'net_position' : np.sum, 
                                 'net_cash' : np.sum, 
                                 'direction': len, 
                                 'id' : lambda x : len(np.unique(x))}).reset_index()
agg = agg.rename(columns = {'net_position': 'net_position_sum', 
                            'net_cash': 'net_cash_sum', 
                            'id': 'n_unique_orders', 
                            'direction': 'n_fills'})
agg.head()

,account,net_position_sum,n_fills,net_cash_sum,n_unique_orders
0,AES15809135,520,29,-16042.00,25
1,BAC86553924,500,25,-15379.60,25
2,BAP89795895,500,25,-15401.60,25
3,BB88003812,500,25,-15487.20,25
4,BFB35565704,10633,452,-278171.34,378


We approximate the last price by taking the price on the last row of our dataframe. Using this, we can compute the profit. We sort by the profit and display the top 20 accounts.

In [5]:
last_price = df.iloc[-1]['price']/100.0 #approximation
agg['profit'] = last_price*agg['net_position_sum'] + agg['net_cash_sum']
agg = agg.sort(['profit'], ascending = 0)
agg.head(n = 20)

,account,net_position_sum,n_fills,net_cash_sum,n_unique_orders,profit
27,HAP46412644,-443180,13870,12428638.360002,454,12303883.190002
61,OBB56644128,-150088,515,4412602.600000,173,4370352.828000
52,MFB35901137,-144979,15530,3622609.509999,377,3581797.921499
14,DLB13864787,-1939,453,95875.180000,383,95329.351500
33,HS16987233,-1746,484,72344.510000,380,71853.011000
25,HAK29172937,-183,427,65642.830000,373,65591.315500
32,HPH49493909,-1561,437,58045.600000,395,57606.178500
49,LOB17361451,-33,651,43023.010000,394,43013.720500
12,DAG98255232,-118,455,42247.510000,392,42214.293000
83,TKB17017295,42,1,-1427.580000,1,-1415.757000


We see that only about 10 accounts are profitable. The two most profitable accounts have massive amounts of fills. The third one is interesting, as it has quite few fills. We compute some key ratios to investigate this further.

In [6]:
agg['profit_per_order'] = agg['profit']/agg['n_unique_orders']
agg['profit_per_fill'] = agg['profit']/agg['n_fills']
agg = agg.sort(['profit_per_fill'], ascending = 0)
agg.head()

,account,net_position_sum,n_fills,net_cash_sum,n_unique_orders,profit,profit_per_order,profit_per_fill
61,OBB56644128,-150088,515,4412602.600000,173,4370352.828000,25262.155075,8486.121996
27,HAP46412644,-443180,13870,12428638.360002,454,12303883.190002,27101.064295,887.086027
52,MFB35901137,-144979,15530,3622609.509999,377,3581797.921499,9500.790243,230.637342
14,DLB13864787,-1939,453,95875.180000,383,95329.351500,248.901701,210.440070
25,HAK29172937,-183,427,65642.830000,373,65591.315500,175.848031,153.609638


We see that one account (OBB56644128) is a clear outlier in terms of the profit_per_fill. This is most likely the inside trader. For a final exercise, we perform outlier detection on our dataset of profitable accounts.

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
detection_set = agg[agg['profit']>0].copy()
xtrain = detection_set.drop(['account'], 1).values
xtrain = StandardScaler().fit_transform(xtrain)
detector = OneClassSVM(random_state = 42).fit(xtrain)
detection_set['label'] = detector.predict(xtrain)
detection_set['decision_function'] = np.round(detector.decision_function(xtrain), 3)
detection_set.sort(['decision_function'], ascending = 1)

,account,net_position_sum,n_fills,net_cash_sum,n_unique_orders,profit,profit_per_order,profit_per_fill,label,decision_function
61,OBB56644128,-150088,515,4412602.600000,173,4370352.828000,25262.155075,8486.121996,-1,-0.699
27,HAP46412644,-443180,13870,12428638.360002,454,12303883.190002,27101.064295,887.086027,-1,-0.674
52,MFB35901137,-144979,15530,3622609.509999,377,3581797.921499,9500.790243,230.637342,-1,-0.299
25,HAK29172937,-183,427,65642.830000,373,65591.315500,175.848031,153.609638,1,0.000
32,HPH49493909,-1561,437,58045.600000,395,57606.178500,145.838427,131.821919,1,0.000
12,DAG98255232,-118,455,42247.510000,392,42214.293000,107.689523,92.778666,1,0.002
49,LOB17361451,-33,651,43023.010000,394,43013.720500,109.171879,66.073303,1,0.005
33,HS16987233,-1746,484,72344.510000,380,71853.011000,189.086871,148.456634,1,0.008
14,DLB13864787,-1939,453,95875.180000,383,95329.351500,248.901701,210.440070,1,0.009


There are not too many profitable accounts but three emerge as outliers. The "sophisticated", highly profitable investors ("HAP46412644" and "MFB35901137"), and our likely inside trader ("OBB56644128").

Now we can submit our solution:

In [9]:
import json
import requests
guilty_account = 'OBB56644128'
explanation_link = 'https://github.com/wallinm1/stockfighter/'
executive_summary = """The error messages for the order cancelling API leak 
some information which allow for the account names to be determined. 
Using account names, executions can be streamed using the websocket-API 
which does not require extra authentication. Using the websocket-API, executions 
were streamed to a text file. From this text file, some key statistics were computed 
for the accounts which were used to determine the outlier trading account."""
data = json.dumps({'account': guilty_account,
                   'explanation_link': explanation_link,
                   'executive_summary': executive_summary.replace('\n', '')})
#insert your id here
instance_id = ''
url = "https://www.stockfighter.io/gm/instances/{}/judge".format(instance_id)
#read api key from file
API_KEY = open('api', 'r').read().strip()
headers = { 'X-Starfighter-Authorization': API_KEY }
#uncomment these
#r = requests.post(url, headers=headers, data=data)
#response = r.json()
#print response, r.status_code
